# Import

In [30]:
# Import de pandas pour manipuler des DataFrames
import pandas as pd   

# Import de ast pour convertir des chaînes JSON en objets Python
import ast            

# Import de os pour gérer les chemins de fichiers
import os   

# Import de train_test_split pour diviser les données en ensembles d'entraînement et de test
from sklearn.model_selection import train_test_split

# Import de Counter pour compter les occurrences
from collections import Counter

# Paths & reading data

In [31]:
# Définition du dossier contenant les fichiers CSV
DATA_DIR = "../../Data"  

# Lecture du fichier movies_metadata.csv
movies = pd.read_csv(os.path.join(DATA_DIR, "movies_metadata.csv"), low_memory=False)

# Lecture du fichier credits.csv
credits = pd.read_csv(os.path.join(DATA_DIR, "credits.csv"))

# Lecture du fichier keywords.csv
keywords = pd.read_csv(os.path.join(DATA_DIR, "keywords.csv"))

In [32]:
# Sélection des colonnes importantes dans le DataFrame movies
movies = movies[[
    "id", "title", "genres", "runtime", "budget",
    "vote_average", "vote_count", "popularity"
]]

# Conversion de la colonne id en numérique, erreurs deviennent NaN
movies["id"] = pd.to_numeric(movies["id"], errors="coerce")

# Conversion de la colonne budget en numérique, erreurs deviennent NaN
movies["budget"] = pd.to_numeric(movies["budget"], errors="coerce")

# Conversion de la colonne runtime en numérique, erreurs deviennent NaN
movies["runtime"] = pd.to_numeric(movies["runtime"], errors="coerce")

# Suppression des lignes sans id ou titre
movies = movies.dropna(subset=["id", "title"])

# Conversion de la colonne id en entier
movies["id"] = movies["id"].astype(int)

In [33]:
# Définition d'une fonction pour extraire les valeurs d'une liste JSON
def parse_json_list(json_str, key):
    # Essaie de convertir la chaîne JSON en liste Python
    try:
        data = ast.literal_eval(json_str)
        # Retourne la liste des valeurs correspondant à la clé 'key'
        return [item[key] for item in data]
    # Si erreur, retourne une liste vide
    except:
        return []

In [34]:
# Extraction des noms des genres depuis la colonne genres
movies["genres"] = movies["genres"].apply(lambda x: parse_json_list(x, "name"))

# Extraction des noms des acteurs depuis la colonne cast
credits["actors"] = credits["cast"].apply(lambda x: parse_json_list(x, "name"))

# Extraction des noms de la crew depuis la colonne crew
credits["crew"] = credits["crew"].apply(lambda x: parse_json_list(x, "name"))

# Extraction des mots-clés depuis la colonne keywords
keywords["keywords"] = keywords["keywords"].apply(lambda x: parse_json_list(x, "name"))

In [35]:
# Fusion du DataFrame movies avec credits sur la colonne id
df = movies.merge(
    credits[["id", "actors", "crew"]],
    on="id",
    how="left"  # Garde tous les films même s'ils n'ont pas de crédits
)

# Fusion du DataFrame précédent avec keywords sur la colonne id
df = df.merge(
    keywords[["id", "keywords"]],
    on="id",
    how="left"  # Garde tous les films même sans mots-clés
)

In [36]:
# Création d'une note pondérée en fonction de la moyenne et du nombre de votes
df["weighted_rating"] = (
    df["vote_average"] * df["vote_count"]
) / (df["vote_count"] + 100)  # +100 pour limiter l'effet des petits votes

In [ ]:
# Séparation du DataFrame en train (80%) et test (20%)
train_df, test_df = train_test_split(
    df, test_size=0.2, random_state=42
)

# Export des DataFrames en CSV
train_df.to_csv(os.path.join(DATA_DIR, "train_data.csv"), index=False)
test_df.to_csv(os.path.join(DATA_DIR, "test_data.csv"), index=False)

print("Export CSV terminé :")
print(" -", os.path.join(DATA_DIR, "train_data.csv"))
print(" -", os.path.join(DATA_DIR, "test_data.csv"))


In [38]:
# Création d'un compteur de popularité pour les acteurs
actor_popularity = Counter()

# Mise à jour du compteur avec les acteurs du train set
for actors in train_df["actors"]:
    actor_popularity.update(actors)

In [39]:
# Création d'un compteur de popularité pour le crew
crew_popularity = Counter()

# Mise à jour du compteur avec le crew du train set
for crew in train_df["crew"]:
    crew_popularity.update(crew)

In [40]:
# Définition d'une fonction pour calculer des features de popularité
def popularity_features(names, popularity_dict):
    # Si ce n'est pas une liste ou si la liste est vide, retourne 0
    if not isinstance(names, list) or len(names) == 0:
        return 0.0, 0.0, 0
    # Récupération de la popularité pour chaque nom
    pops = [popularity_dict.get(name, 0) for name in names]
    # Retourne : moyenne, max et nombre de personnes "connues"
    return (
        sum(pops) / len(pops),   # Popularité moyenne
        max(pops),               # Popularité max
        sum(p > 5 for p in pops) # Nombre de personnes connues
    )

In [41]:
# Calcul des features de popularité pour les acteurs du train set
train_df[["actor_pop_mean", "actor_pop_max", "actor_pop_known"]] = train_df["actors"].apply(
    lambda x: pd.Series(popularity_features(x, actor_popularity))
)

# Calcul des features de popularité pour les acteurs du test set
test_df[["actor_pop_mean", "actor_pop_max", "actor_pop_known"]] = test_df["actors"].apply(
    lambda x: pd.Series(popularity_features(x, actor_popularity))
)

In [42]:
# Calcul des features de popularité pour la crew du train set
train_df[["crew_pop_mean", "crew_pop_max", "crew_pop_known"]] = train_df["crew"].apply(
    lambda x: pd.Series(popularity_features(x, crew_popularity))
)

# Calcul des features de popularité pour la crew du test set
test_df[["crew_pop_mean", "crew_pop_max", "crew_pop_known"]] = test_df["crew"].apply(
    lambda x: pd.Series(popularity_features(x, crew_popularity))
)

In [43]:
# Affichage de statistiques descriptives des nouvelles features de popularité
train_df[[
    "actor_pop_mean", "actor_pop_max", "actor_pop_known",
    "crew_pop_mean", "crew_pop_max", "crew_pop_known"
]].describe()

,actor_pop_mean,actor_pop_max,actor_pop_known,crew_pop_mean,crew_pop_max,crew_pop_known
count,37300.000000,37300.000000,37300.000000,37300.000000,37300.000000,37300.000000
mean,10.470147,31.556005,5.783164,9.653932,26.166676,4.542708
std,8.635252,26.485943,6.254640,9.759163,32.159583,7.160077
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.750000,10.000000,1.000000,3.000000,5.000000,0.000000
50%,8.500000,27.000000,4.000000,6.647059,14.000000,2.000000
75%,15.357143,47.000000,8.000000,12.928571,34.000000,6.000000
max,121.000000,196.000000,140.000000,122.000000,214.000000,90.000000
